<a href="https://colab.research.google.com/github/mckang1020/deep-learning-from-scratch-practice/blob/master/Chapter6_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 제 6장. 순환 신경망 RNN (Recurrent Neural Network)



In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [0]:
n_hidden = 35
lr = 0.01
epochs = 100

string = "hello pytorch. how long can a rnn cell remember?"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
n_letters = len(char_list)


*   어떤 문장이 들어왔을 때 맨 앞에 시작 토큰(start token)과 맨 뒤에 끝 토큰(end token)을 붙이고, 원-핫 벡터로 변환하여 전달하는 함수 생성  
*   반대로 원-핫 벡터를 단어로 바꿔주는 함수 생성

*   문자를 그대로 쓰지않고 one-hot 벡터로 바꿔서 연산에 쓰도록 하겠습니다.

Start = [0 0 0 … 1 0] <br/> 
　　a = [1 0 0 … 0 0] <br/>
　　b =     [0 1 0 … 0 0] <br/>
　　c =     [0 0 1 … 0 0] <br/>
　　　... <br/>
　end =   [0 0 0 … 0 1]



In [0]:
def string_to_onehot(string):
  start = np.zeros(shape=len(char_list), dtype=int)
  end = np.zeros(shape=len(char_list), dtype=int)
  start[-2] = 1
  end[-1] = 1
  for i in string:
    idx = char_list.index(i)  
    zero = np.zeros(shape=n_letters, dtype=int)  # 0으로 구성된 배열을 생성
    zero[idx] = 1                                # 해당 string에 대응하는 0에 1을 할당 (본인 자리만 1)
    start = np.vstack([start, zero])  # 수직으로 (vertical) 쌓아 합치는 함수, start 아래에 문자열 관련 벡터를 쌓음
  output = np.vstack([start, end])  # 문자열이 다 끝나면 쌓아온 start에 end를 추가
  return output

In [0]:
def onehot_to_word(onehot_1):
  onehot = torch.Tensor.numpy(onehot_1)  # 텐서 -> 넘파이 배열로 변환
  return char_list[onehot.argmax()]  # 원소 중 가장 큰 값(여기선 1) 인덱스를 뽑고 -> char_list에서 찾음



*   class 형태의 모델은 항상 `nn.Module` 을 상속받아야 하며, `super(모델명, self).__init__()` 을 실행시키는 코드가 필요


In [0]:
# 은닉층이 하나인 RNN

class RNN(nn.Module):  # ◆ 상속 개념?
  def __init__(self, input_size, hidden_size, output_size):
    super(RNN, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size

    self.i2h = nn.Linear(input_size, hidden_size)  # affine operation 층
    self.h2h = nn.Linear(hidden_size, hidden_size)
    self.i2o = nn.Linear(hidden_size, output_size)
    self.act_fn = nn.Tanh()

  def forward(self, input, hidden):
    hidden = self.act_fn(self.i2h(input)+self.h2h(hidden))
    output = self.i2o(hidden)
    return output, hidden

  def init_hidden(self): 
    return torch.zeros(1, self.hidden_size)

rnn = RNN(n_letters, n_hidden, n_letters)

# 1. 상속 (Inheritance)

출처: [파이썬-기본을 갈고 닦자 / 상속](https://wikidocs.net/16073)

* 클래스에서 상속이란, 물려주는 클래스(*parent class, super class*)의 내용(*속성과 메소드*)을 물려받는 클래스(*child class, sub class*)가 가지게 되는 것

```
class 부모클래스:
  ...내용...

class 자식클래스(부모클래스):
  ...내용...
```

# 2. 메소드 오버라이딩 (Method overriding)

2-1. 일반적인 메소드 오버라이딩
* 메소드 오버라이딩은 부모 클래스의 메소드를 자식 클래스에서 재정의하는 것

2-2 부모 메소드'도' 호출하는 오버라이딩
* 부모클래스의 메소드도 수행하고, 자식클래스의 메소드의 내용도 함께 출력하고 싶은 경우
* 그럴때 `super()` 라는 키워드를 사용하면 자식클래스 내에서 코드에서도 부모클래스를 호출할 수 있음

```
class Country:
    """Super Class"""

    name = '국가명'
    population = '인구'
    capital = '수도'

    def show(self):
        print('국가 클래스의 메소드입니다.')

class Korea(Country):
    """Sub Class"""

    def __init__(self, name,population, capital):
        self.name = name
        self.population = population
        self.capital = capital
    
    def show(self):
        super().show()
        print(
            """
            국가의 이름은 {} 입니다.
            국가의 인구는 {} 입니다.
            국가의 수도는 {} 입니다.
            """.format(self.name, self.population, self.capital)
        )
```

```
>>> from inheritance import *
>>> a = Korea('대한민국', 50000000, '서울')
>>> a.show()
국가 클래스의 메소드입니다.

            국가의 이름은 대한민국 입니다.
            국가의 인구는 50000000 입니다.
            국가의 수도는 서울 입니다.

>>> 
```

# 3. 다중상속 (Mutiple Inheritance)

* 여러 기반 클래스로부터 상속을 받아서 파생 클래스를 만드는 방법
* `C#` 또는 `Java` 는 다중상속이 불가능한 언어
* `Python` 과 `C++` 에서는 가능

```
class 부모클래스1:
    ...내용...

class 부모클래스2:
    ...내용...    

class 자식클래스(부모클래스1, 부모클래스2):
    ...내용...
```
* 다중상속을 이용하여 다음과 같은 상속도 가능
* `class D` 에서 `class A` 메소드 사용 가능 
* 그러나 B 와 C 중 경우에 따라 호출하는 것이 달라진다면 문제가 발생할 수 있어 바람직하지 않음

```
class A:
    def greeting(self):
        print('안녕하세요. A입니다.')  

class B(A):
    def greeting(self):
        print('안녕하세요. B입니다.')

class C(A):
    def greeting(self):
        print('안녕하세요. C입니다.')

class D(B, C):
    pass
 
>>> x = D()
>>> x.greeting()
안녕하세요. B입니다.

```

![다이아몬드 상속](https://drive.google.com/uc?id=1MUarwisAEHy1fwD3xoqMlSgtc19pl2sk)

# 4. 메서드 탐색 순서 확인하기
* 많은 프로그래밍 언어들이 다이아몬드 상속에 대한 해결책을 제시
* `Python` 에서는 메서드 탐색 순서(Method Resolution Order, MRO) 를 따름
* `클래스.mro()` 로 사용

```
>>> D.mro()
[<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>]
```
* D 로 인스턴스를 만들고 greeting을 호출하면 B의 greeting 이 호출됨 (D에는 greeting 메서드가 없으므로)
* `class D(B, C)` 에서 왼쪽에서 오른쪽 순서로 메서드를 찾음

In [0]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [0]:
torch.ones(1)
torch.ones(1,1)
torch.ones(1,1,1)

In [0]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())  # ◆ `FloatTensor`는 함수? 데이터형이 들어가는 것 아님?

for i in range(epochs):
  rnn.zero_grad()  # ◆ 아래 설명 참고
  total_loss = 0
  hidden = rnn.init_hidden()  # 초기화

  for j in range(one_hot.size()[0]-1):  # 입력은 앞에 글자 pythorch 에서 p y t o r c h
    input_ = one_hot[j:j+1,:]  # ◆ one_hot[j+1,:] 이렇게 하면 안되나? 이렇게 하는 이유? -> 매트릭스 화
    target = one_hot[j+1]

    output, hidden = rnn.forward(input_, hidden)
    loss = loss_func(output.view(-1), target.view(-1))
    total_loss += loss
    input_ = output  # ◆ 변수명에 '_'는 왜 붙이는 것? in-place? 함수에만 붙이는 것 아니였나? 명시적이기 위해?

  total_loss.backward()
  optimizer.step()

  if i % 10 == 0:
    print(total_loss)


tensor(2.6027, grad_fn=<AddBackward0>)
tensor(1.0020, grad_fn=<AddBackward0>)
tensor(0.6796, grad_fn=<AddBackward0>)
tensor(0.4723, grad_fn=<AddBackward0>)
tensor(0.3244, grad_fn=<AddBackward0>)
tensor(0.2246, grad_fn=<AddBackward0>)
tensor(0.2080, grad_fn=<AddBackward0>)
tensor(0.1485, grad_fn=<AddBackward0>)
tensor(0.1207, grad_fn=<AddBackward0>)
tensor(0.1006, grad_fn=<AddBackward0>)


* [`rnn.zero_grad()`](https://discuss.pytorch.org/t/zero-grad-optimizer-or-net/1887) 에 대한 설명
* 파이토치에서는 backward를 할때 gradients를 축적하기 때문에 매번 0으로 바꿔주어야 한다. 
![결국 같다](https://drive.google.com/uc?id=1L75xuuWMGZhww68h9CJvbcLsFmb8-cda)

![opt.zero_grad() vs model.zero_grad()](https://drive.google.com/uc?id=1-cWzK2PNNuE-mC_Zy_xGRixLs6cAVszA)

In [0]:
start = torch.zeros(1,len(char_list))
start[:,-2] = 1

with torch.no_grad():   # ◆ 아래 설명 참고
  hidden = rnn.init_hidden()
  input_ = start        # 첫 입력값은 start token
  output_string = ""    # 결과로 나오는 문자들을 계속 붙여준다
  for i in range(len(string)):
    output, hidden = rnn.forward(input_, hidden)
    output_string += onehot_to_word(output.data)  # x가 variable 일 때, x.data 는 그 값을 갖는 tensor
    input_ = output

print(output_string)

helloreytoec r n r ron an n r r n  n  c c   n  n


# 1. try - finally 블록
* 파일의 작업 흐름은 `open()` 내장함수로 파일을 열고, 파일 객체를 통해 파일을 작업하고, 파일 객체의 `close()` 함수로 파일을 닫아야 함
* 파일을 닫지 않으면 데이터가 소실될 수 있음
* try - finally 절로 묶는 이유는 예외가 발생을 해도 finally 절을 통해 반드시 파일을 close 시킬 수 있기 때문

```
try:
  변수 = open(파일경로, 옵션)
  ... 파일 조작 ...
finally:
  변수.close()
``` 
<br>

# 2. with 블록
* 같은 작업이지만, with 블록을 통해 명시적으로 `close()` 메소드를 호출하지 않고도 파일을 닫을 수 있음
* with 블록이 자동으로 블록을 종료할 때 `__exit()__` 메소드를 호출하여 파일을 close 시키게 됨

<br>

# 3. 따라서 `with torch.no_grad()` 의 의미는?
출처: [tutorials.pytorch.kr](https://tutorials.pytorch.kr/beginner/blitz/autograd_tutorial.html)

> `torch.Tensor` 클래스에서 `.requires_grad` 속성을 `True` 로 설정하면, 그 tensor에서 이뤄진 모든 연산들을 추적(track)하기 시작한다. <br><br> 계산이 완료된 후 `.backward()` 를 호출하여 모든 변화도(gradients)를 자동으로 계산할 수 있다. 이 tensor의 변화도는 `.grad` 속성에 누적된다.<br><br> tensor가 기록하는 것을 중단하게 하려면, `.detach()` 를 호출하여 연산 기록으로부터 분리(detach)하여 이후 연산들이 추적되는 것을 방지할 수 있다.

* 기록을 추적하는 것(과 메모리를 사용하는 것)을 방지하지 위해, 코드 블록을 `with torch.no_grad():` 로 감쌀 수 있다. 이는 특히 변화도(gradient)는 필요없지만, `requires_grad = True` 가 설정되어 학습 가능한 매개변수를 갖는 모델을 평가(evaluate)할 때 유용하다.
<br><br>
---



# 코드 차이점 비교 분석 (책 vs. colab)

![RNN 계층 layer 비교](https://drive.google.com/uc?id=1KXGE9FcuVY5a5p8au47iSiY-Q2MbEY1K)

* 왼쪽 코드를 (1)번, 오른쪽 코드를 (2)번이라고 하자
* 1번에서 `i2h`, `h2h` 계층은 RNN 아키텍쳐 상 이해가 가지만, `i2o` 에는 동의하지 못하겠음
* hidden layer에서 output layer이면 `h2o` 이 더 자연스러운듯 함
<br><br>
* 1번에서는 일반적인 RNN 계층 처럼 input값과 hidden값을 더해 `i2o` 에 넣어 output를 도출하고, return 값으로 나온 그 output값과 사용되었던 hidden값을 다시 다음 글자를 맞추는데(output를 도출) 사용
* 그러나, 2번에서는 input값과 hidden값을 병합시켰음(물론 크기도 `nn.Linear(input_size + hidden_size)` 로 맞춰줌)
* 이유는??? -> 시퀀스 특성을 보존하기 위해

![concat layer](https://drive.google.com/uc?id=1frG3EHKg6Wvm9XfDgieTMhi9FlAnFD0p)

---

# LSTM (Long short-term memory)

![LSTM](https://drive.google.com/uc?id=1VhcVvNMn8IzQ4yR-DoBpleoCAYWVuzVQ)

* 기존의 순환 신경망 모델에 장기기억<sup>long-term memory</sup>을 담당하는 부분을 추가한 것
* 기존에 있던 은닉 상태<sup>hidden state</sup>에 셀 상태<sup>cell state</sup>를 추가함
* 망각 게이트와 입력 게이트를 통해 셀 상태의 값을 얼마나 잊어버릴지 또는 기억할지를 조절
* 다시 말해, 현재 시점의 새로운 입력값과 직전 시점의 은닉 상태의 값의 조합으로 기존의 셀 상태의 정보를 얼마큼 전달할지도 정하고 어떤 정보를 얼마만큼의 비중을 더할지도 정하는 것

★ tanh 함수 쓰는 이유 ReLU 안되는 이유?
[설명](https://stackoverflow.com/questions/40761185/what-is-the-intuition-of-using-tanh-in-lstm)




### 1) 예시

파라미터 | 하는 일
--- | ---
input_size | 입력의 특성 개수
hidden_size | hidden state의 특성 개수
num_layers | LSTM을 몇층으로 쌓을것인가 여부
bias | 편차의 사용 여부
batch_first | 사용하면 입력과 출력의 형태가 [batch, seq, feature]
dropout | 드롭아웃 사용여부
bidirectional | 양방향 LSTM 사용여부 (기본이 False)


In [0]:
import torch
import torch.nn as nn

rnn = nn.LSTM(input_size=3, hidden_size=5, num_layers=2)

# ◆ 위에서와 같은 질문 왜 input_ 에서 _를 붙이는 거지?
# 기본적으로 입력의 형태는 (seq_len, batch, input_size)
input_ = torch.randn(5, 3, 3)  

# '히든'과 '셀'의 형태는 (num_layer * num_directions, batch, hidden_size) 
# ◆ num_directions는 bidirectional 일때 2, 그렇다면 어차피 그 값은 1 또는 2 ?
h0 = torch.randn(2, 3, 5)
c0 = torch.randn(2, 3, 5)

# LSTM에 입력을 전달할때는 동일하게 input, (h_0, c_0) 처럼 상태를 튜플로 묶어서 전달
output, (hidden_state, cell_state) = rnn(input_, (h0, c0)) 

print(output.size(),hidden_state.size(),cell_state.size())

torch.Size([5, 3, 5]) torch.Size([2, 3, 5]) torch.Size([2, 3, 5])


### 2) 하드 코딩
* batch_first를 사용, 즉 입력과 출력의 형태가 `[batch, seq, feature]`

In [0]:
rnn = nn.LSTM(3, 5, 2, batch_first=True)

# batch_first를 사용하면 입력의 형태는 (batch, seq, feature)
input_ = torch.randn(3, 5, 3)

# '히든'과 '셀'의 형태는 동일하게 (num_layers * num_directions, batch, hidden_size)
h0 = torch.randn(2, 3, 5)
c0 = torch.randn(2, 3, 5)

# LSTM에 입력을 전달할때는 동일하게 input, (h_0, c_0) 처럼 상태를 튜플로 묶어서 전달
output, (hidden_state, cell_state) = rnn(input_, (h0, c0))

print(input_.size(),h0.size(),c0.size())
print(output.size(),hidden_state.size(),cell_state.size())  # ◆★ 인풋의 feature는 3인데, 아웃풋의 feature는 5 ..?

torch.Size([3, 5, 3]) torch.Size([2, 3, 5]) torch.Size([2, 3, 5])
torch.Size([3, 5, 5]) torch.Size([2, 3, 5]) torch.Size([2, 3, 5])


### 3) 하이퍼 파라미터로 표현

In [0]:
input_size = 7
batch_size = 4
hidden_size = 3
seq_len = 2

rnn = nn.LSTM(input_size, hidden_size, seq_len, batch_first=True)

input_ = torch.randn(batch_size, hidden_size, input_size)
h0 = torch.randn(seq_len, batch_size, hidden_size)
c0 = torch.randn(seq_len, batch_size, hidden_size)

output, (hidden_state, cell_state) = rnn(input_, (h0, c0))

print(input_.size(),h0.size(),c0.size())
print(output.size(),hidden_state.size(),cell_state.size())

torch.Size([4, 3, 7]) torch.Size([2, 4, 3]) torch.Size([2, 4, 3])
torch.Size([4, 3, 3]) torch.Size([2, 4, 3]) torch.Size([2, 4, 3])


---

### 같은 task를 `RNN`에서 `LSTM`으로 구현

In [0]:
string = "hello pytorch. how long can a rnn cell remember? show me your limit!"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
char_len = len(char_list)

# 하이퍼파라미터 설정
# 문자열을 단어 하나씩 잘러서 사용하는걸로 구현하여 batch_size 1로 고정
# batch_size가 1보다 큰 경우는 다음 실습코드
batch_size = 1

# seq_len는 바꿔도 학습은 되지만 테스트시 편의성을 위해 1로 설정
# ◆ sequence length 를 의미? 
seq_len = 1

# num_layers는 입력 형식에만 맞게 형태를 바꿔주면 됩니다.
num_layers = 3
input_size = char_len
hidden_size = 35 
lr = 0.01
num_epochs = 1000

one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

print(one_hot.size())

torch.Size([70, 35])


In [0]:
# RNN with 1 hidden layer

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size,num_layers):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers)
        
    def forward(self,input_,hidden,cell):
        output,(hidden,cell) = self.lstm(input_,(hidden,cell))
        return output,hidden,cell
    
    def init_hidden_cell(self):
        hidden = torch.zeros(num_layers,batch_size,hidden_size)
        cell = torch.zeros(num_layers,batch_size,hidden_size)
        return hidden,cell
    
rnn = RNN(input_size,hidden_size, num_layers)

In [0]:
# Loss function & Optimizer

loss_func = nn.MSELoss()  # ★ 크로스 엔트로피 안쓰는 이유?
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [0]:
j=0
input_data = one_hot[j:j+seq_len].view(seq_len, batch_size, input_size)
print(input_data.size())

hidden,cell = rnn.init_hidden_cell()
print(hidden.size(),cell.size())

output, hidden,cell = rnn(input_data,hidden,cell)
print(output.size(),hidden.size(),cell.size())

torch.Size([1, 1, 35])
torch.Size([3, 1, 35]) torch.Size([3, 1, 35])
torch.Size([1, 1, 35]) torch.Size([3, 1, 35]) torch.Size([3, 1, 35])


In [0]:
unroll_len = one_hot.size()[0]//seq_len -1
for i in range(num_epochs):
    hidden,cell = rnn.init_hidden_cell()
    
    loss = 0
    for j in range(unroll_len):
        input_data = one_hot[j:j+seq_len].view(seq_len,batch_size,input_size) 
        label = one_hot[j+1:j+seq_len+1].view(seq_len,batch_size,input_size)
        
        optimizer.zero_grad()
        
        output, hidden, cell = rnn(input_data,hidden,cell)
        loss += loss_func(output.view(1,-1),label.view(1,-1))
        
    loss.backward()
    optimizer.step()

    if i%10 ==0:
        print(loss)

tensor(2.3196, grad_fn=<AddBackward0>)
tensor(1.8022, grad_fn=<AddBackward0>)
tensor(1.6776, grad_fn=<AddBackward0>)
tensor(1.5151, grad_fn=<AddBackward0>)
tensor(1.2524, grad_fn=<AddBackward0>)
tensor(0.8537, grad_fn=<AddBackward0>)
tensor(0.4916, grad_fn=<AddBackward0>)
tensor(0.2358, grad_fn=<AddBackward0>)
tensor(0.1195, grad_fn=<AddBackward0>)
tensor(0.0725, grad_fn=<AddBackward0>)
tensor(0.0519, grad_fn=<AddBackward0>)
tensor(0.0347, grad_fn=<AddBackward0>)
tensor(0.0258, grad_fn=<AddBackward0>)
tensor(0.0199, grad_fn=<AddBackward0>)
tensor(0.0161, grad_fn=<AddBackward0>)
tensor(0.0138, grad_fn=<AddBackward0>)
tensor(0.0124, grad_fn=<AddBackward0>)
tensor(0.0114, grad_fn=<AddBackward0>)
tensor(0.0108, grad_fn=<AddBackward0>)
tensor(0.0104, grad_fn=<AddBackward0>)
tensor(0.0099, grad_fn=<AddBackward0>)
tensor(0.0094, grad_fn=<AddBackward0>)
tensor(0.0091, grad_fn=<AddBackward0>)
tensor(0.0093, grad_fn=<AddBackward0>)
tensor(0.0088, grad_fn=<AddBackward0>)
tensor(0.0085, grad_fn=<A

In [0]:
hidden,cell = rnn.init_hidden_cell()

for j in range(unroll_len-1):
    input_data = one_hot[j:j+1].view(1,batch_size,hidden_size) 
    label = one_hot[j+1:j+1+1].view(1,batch_size,hidden_size) 
    
    output, hidden, cell = rnn(input_data,hidden,cell)
    print(onehot_to_word(output.data),end="") 

hello pytorch. how long can a rnn cell remember? show me your limit!

 ★ 1. RNN 크기 관련 그림
    2. RNN에서 MSE Loss 를 쓰는 이유

---

### LSTM: batch size를 3으로 증가

In [0]:
import torch 
import torch.nn as nn
import torch.optim as optim
import numpy as np

string = "hello pytorch. how long can a rnn cell remember? show me your limit!"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
char_len = len(char_list)

def string_to_onehot(string):
    start = np.zeros(shape=char_len ,dtype=int)
    end = np.zeros(shape=char_len ,dtype=int)
    start[-2] = 1
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape=char_len ,dtype=int)
        zero[idx]=1
        start = np.vstack([start,zero])
    output = np.vstack([start,end])
    return output

def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

# 하이퍼파라미터 설정
# 이번코드는 배치사이즈가 1보다 큰 경우
batch_size = 5

# seq_len는 바꿔도 학습은 되지만 테스트시 편의성을 위해 1로 설정
seq_len = 1

# num_layers는 자유롭게 바꿀 수 있음
num_layers = 3
input_size = char_len
hidden_size = 35 
lr = 0.01
num_epochs = 1000

one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size,num_layers):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
    def forward(self,input,hidden,cell):
        output,(hidden,cell) = self.lstm(input,(hidden,cell))
        return output,hidden,cell
    
    def init_hidden_cell(self):
        hidden = torch.zeros(num_layers, batch_size, hidden_size)
        cell = torch.zeros(num_layers, batch_size, hidden_size)
        return hidden,cell
    
rnn = RNN(input_size,hidden_size, num_layers)

# Loss function & Optimizer
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

j=0
input_data = one_hot[j:j+batch_size].view(batch_size,seq_len,input_size)
print(input_data.size())

hidden,cell = rnn.init_hidden_cell()
print(hidden.size(),cell.size())

output,hidden,cell = rnn(input_data,hidden,cell)
print(output.size(),hidden.size(),cell.size())

unroll_len = one_hot.size()[0]//seq_len -1
for i in range(num_epochs):
    optimizer.zero_grad()
    hidden,cell = rnn.init_hidden_cell()
    
    loss = 0
    for j in range(unroll_len-batch_size+1):
        
        # batch size에 맞게 one-hot 벡터를 스택
        # 예를 들어 batch size가 3이면 pytorch에서 pyt를 one-hot 벡터로 바꿔서 쌓고 목표값으로 yto를 one-hot 벡터로 바꿔서 쌓는 과정
        input_data = torch.stack([one_hot[j+k:j+k+seq_len] for k in range(batch_size)],dim=0)
        label = torch.stack([one_hot[j+k+1:j+k+seq_len+1] for k in range(batch_size)],dim=0)
        
        input_data = input_data
        label = label
        
        output, hidden, cell = rnn(input_data,hidden,cell)
        loss += loss_func(output.view(1,-1),label.view(1,-1))
        
    loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(loss)


torch.Size([3, 1, 35])
torch.Size([3, 3, 35]) torch.Size([3, 3, 35])
torch.Size([3, 1, 35]) torch.Size([3, 3, 35]) torch.Size([3, 3, 35])
tensor(2.2764, grad_fn=<AddBackward0>)
tensor(1.7482, grad_fn=<AddBackward0>)
tensor(1.6716, grad_fn=<AddBackward0>)
tensor(1.5531, grad_fn=<AddBackward0>)
tensor(1.3611, grad_fn=<AddBackward0>)
tensor(1.0665, grad_fn=<AddBackward0>)
tensor(0.6247, grad_fn=<AddBackward0>)
tensor(0.3634, grad_fn=<AddBackward0>)
tensor(0.1737, grad_fn=<AddBackward0>)
tensor(0.0981, grad_fn=<AddBackward0>)
tensor(0.0633, grad_fn=<AddBackward0>)
tensor(0.0448, grad_fn=<AddBackward0>)
tensor(0.0344, grad_fn=<AddBackward0>)
tensor(0.0277, grad_fn=<AddBackward0>)
tensor(0.0237, grad_fn=<AddBackward0>)
tensor(0.0210, grad_fn=<AddBackward0>)
tensor(0.0191, grad_fn=<AddBackward0>)
tensor(0.0177, grad_fn=<AddBackward0>)
tensor(0.0190, grad_fn=<AddBackward0>)
tensor(0.0160, grad_fn=<AddBackward0>)
tensor(0.0146, grad_fn=<AddBackward0>)
tensor(0.0138, grad_fn=<AddBackward0>)
tens

In [0]:
hidden,cell = rnn.init_hidden_cell()

for j in range(unroll_len-batch_size+1):
    input_data = torch.stack([one_hot[j+k:j+k+seq_len] for k in range(batch_size)],dim=0)
    label = torch.stack([one_hot[j+k+1:j+k+seq_len+1] for k in range(batch_size)],dim=0)

    input_data = input_data
    label = label
    
    output, hidden, cell = rnn(input_data,hidden,cell)
    for k in range(batch_size):
        print(onehot_to_word(output[k].data),end="")
        if j < unroll_len-batch_size:
            break

hello pytorch. how long can a rnn cell remember? show me your limit!1

* 왜 이렇게 안나오지? 코드 점검
![왜이렇게안나오지](https://drive.google.com/uc?id=144u-phptYRK7s8P23-k9mOr3OnP6eysZ)

---

## 더 큰 데이터에 학습시키기 (feat.셰익스피어)

# 1. Naive RNN
* 셰익스피어 문체를 모방하는 순환신경망 실습
* 임베딩<sup>Embedding</sup> 레이어 및 RNN 모델로 구성

1-1. 데이터 준비


In [1]:
!rm -r data
import os 

try:
  os.mkdir("./data")  # 폴더 생성 (make directory)
except:
  pass

!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt -P ./data

rm: cannot remove 'data': No such file or directory
--2019-11-29 08:31:55--  https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘./data/input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.09s   

2019-11-29 08:31:56 (11.8 MB/s) - ‘./data/input.txt’ saved [1115394/1115394]



1-2. 필요한 라이브러리 임포트

In [2]:
import torch
import torch.nn as nn

!pip install unidecode

import unidecode
import string
import random
import re
import time, math

     |████████████████████████████████| 245kB 2.8MB/s 


1-3. 하이퍼 파라미터 들 선언

In [0]:
num_epochs = 2000
print_every = 100
plot_every = 10

chunk_len = 200  # 설명은 함수정의에서 함

hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

1-4. 데이터 전처리

In [4]:
all_characters = string.printable  # import 했던 string에서 출력가능한 문자들을 다 불러옴
                          
n_characters = len(all_characters)  # 출력가능한 문자들의 개수를 저장
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


In [5]:
file = unidecode.unidecode(open('./data/input.txt').read())  # text 를 읽어옴
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394


1-5. 등장하는 함수들

1) random chunk

In [6]:
# 이 함수는 텍스트 파일의 일부분을 랜덤하게 불러옴
def random_chunk():
    # (시작지점 < 텍스트파일 전체길이 - 불러오는 텍스트의 길이)가 되도록 시작점과 끝점 설정
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

IANCA:
If you affect him, sister, here I swear
I'll plead for you myself, but you shall have
him.

KATHARINA:
O then, belike, you fancy riches more:
You will have Gremio to keep you fair.

BIANCA:
Is i


2) character to tensor

In [7]:
# 문자열을 받았을때 이를 인덱스의 배열로 바꿔주는 함수
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()  # self.long() is equivalent to self.to(torch.int64)
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


3) chunk into input & label

In [0]:
# 랜덤한 텍스트 chunk를 불러와서 이를 입력과 목표값을 바꿔주는 함수
# ex) pytorch라는 문자열이 들어오면 입력은 pytorc / 목표값은 ytorch (output)
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

1-6. 모델과 옵티마이져

1) 모델

In [0]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.encoder = nn.Embedding(self.input_size, self.embedding_size)  # 임베딩 함수
        self.rnn = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers)
        self.decoder = nn.Linear(self.hidden_size, self.output_size)  # 디코더
        
    
    def forward(self, input, hidden):
        out = self.encoder(input.view(1,-1))
        out,hidden = self.rnn(out,hidden)
        out = self.decoder(out.view(batch_size,-1))
        return out,hidden

    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden
    

model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)

* 원래 실무에서는 word2vec을 미리 만들어두고 사용하지만, 여기서는 임베딩<sup>Embedding</sup> 역시 학습이 되는 방식으로 구현

* `torch.nn.Embedding` 이라는 class에서 `num_embeddings`와 `embedding_dim`은 각각 사용할 문자나 단어의 수 및 임베딩할 벡터 공간의 크기를 의미

> ex) a, b, c, d 의 4가지(`num_embeddings=4`) 문자를 10차원(`embedding_dim=10`)으로 표현하면, 4 x 10 벡터가 생성 되며 각각의 행이 a, b, c, d 를 의미

![embedding포함 RNN](https://drive.google.com/uc?id=1XtOhAVZjx6mH_EVhPlLisFqrQK7I21Ey)


In [0]:
model = RNN(input_size=n_characters, 
            embedding_size=embedding_size,
            hidden_size=hidden_size, 
            output_size=n_characters, 
            num_layers=2)

In [11]:
# 모델 테스트 
# hidden_size = 100
# batch_size = 1
# num_layers = 1  # ◆ 위에서 1로 선언했었는데?
# embedding_size = 70

inp = char_tensor("A")  # 텐서로 변환
print(inp)
hidden = model.init_hidden()
print(hidden.size())
out,hidden = model(inp,hidden)
print(out.size())  # num_chars =  100 이므로 -> output 크기 = 100

tensor([36])
torch.Size([2, 1, 100])
torch.Size([1, 100])


2) 손실 함수와 옵티마이져

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

3) test 함수

* 임의의 문자(start_str)로 시작하는 길이 200짜리 모방 글을 생성

In [0]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden = model(x,hidden)

        # 여기서 max값을 사용하지 않고 multinomial을 사용하는 이유는 
        # 만약 max 값만 쓰는 경우에 생성되는 텍스트가 다 the the the 이런식으로 나오기 때문
        # multinomial 함수를 통해 높은 값을 가지는 문자들중에 램덤하게 다음 글자를 뽑아내는 방식으로 
        # 자연스러운 텍스트를 생성해보자
        output_dist = output.data.view(-1).div(0.8).exp()
        #  ★ 0.8 -> 증폭? exp() ?
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

1-7. 학습

In [15]:
for i in range(num_epochs):
    # 랜덤한 텍스트 덩어리를 샘플링하고 이를 인덱스 텐서로 변환합니다. 
    inp,label = random_training_set()
    hidden = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden = model(x,hidden)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n","="*100)


 tensor([4.6268], grad_fn=<DivBackward0>) 

bMo1FG*.^}(7OUp$aJ]]R
TMii\n"bp
~9~|0

 tensor([2.5591], grad_fn=<DivBackward0>) 

blis, is,or thit
 coul,
Eon hanruw sor stwWist ceive I sminr nos sony sanesd yat.
Wh?t
Why goreturr thr, bow:
Od mas ony rurin he st ny lD bars no nis il houg bhy aner lifasoO matheves woret fcathe, Bn

 tensor([2.3313], grad_fn=<DivBackward0>) 

besy ind:
Ang yarthale, blall beve be thin dilch sorn,
The I ont torullaf iy?
Mensom kopCe urdond thind:
Thend were deeith the ter!
And or te d, will weas, balad:
Houf?

ORAUUR:
Av! I GF Iurce ht your 

 tensor([2.2919], grad_fn=<DivBackward0>) 

bnis Roghise peore bat ay be
Rroord say, by ut; in, thy michirs youn the Hantlink and thay nold not thes this'dis mocer; saem than my rath seer that rornou whanm trithe arnidd wurd masmis frarsy I qed 

 tensor([2.2789], grad_fn=<DivBackward0>) 

brow in nour, tholle not the; hat, hat lore, whune pass hands, he her.
Adland when you heace;
The hit have for he mos bave he cring

---

* GRU 는 다음과 같이 위의 코드에서 RNN 부분을 GRU 로 변경

```
self.rnn = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers)

self.rnn = nn.GRU(self.embedding_size, self.hidden_size, self.num_layers)
```

* LSTM 은 위의 코드에서 `cell`를 추가
![차이점](https://drive.google.com/uc?id=1zYKITpgZa3tCLbTQvV2qBsbt2lOsfFgy)